In [ ]:
!pip install git+https://github.com/PrithivirajDamodaran/Styleformer.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Styleformer.git to /tmp/pip-req-build-xdjzyoic
  Running command git clone -q https://github.com/PrithivirajDamodaran/Styleformer.git /tmp/pip-req-build-xdjzyoic


In [ ]:
from styleformer import Styleformer
import torch
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
style2idx = {
    'Casual to Formal' : 0,
    'Formal to Casual' : 1,
    'Active to Passive' : 2,
    'Passive to Active' : 3
}

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, modified_precision
from tqdm.notebook import tqdm


def tokenize(text):
    tokenized_data = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            tokenized_data.append(word)
    return tokenized_data


def get_bleu_score(reference, candidate):
    reference = [tokenize(reference)]
    candidate = tokenize(candidate)
    bleu_score = float(modified_precision(reference, candidate, n=1))
    return bleu_score, candidate


def transferStyle(text, model):
    sentences = [model.transfer(sentence) for sentence in sent_tokenize(text)]
    print(sentences)
    return ' '.join(filter(None, sentences)).strip(' ')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sf_CasualFormal = Styleformer(style = style2idx['Casual to Formal'])
sf_FormalCasual = Styleformer(style = style2idx['Formal to Casual'])
sf_ActivePassive = Styleformer(style = style2idx['Active to Passive'])
sf_PassiveActive = Styleformer(style = style2idx['Passive to Active'])

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Passive to Active model loaded...


In [ ]:
# Before you run this, make sure that the shared CSE 573: Stance Detection folder is on your My Drive tab. You can do this by right clicking on the folder and selecting "Add shortcut to Drive"
from google.colab import drive
from google.colab import files
from tqdm.notebook import tqdm
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Progress Bar
tqdm().pandas()


usingProcon = False

if usingProcon:
  proconfile_train = open('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Procon20/procon_final.csv', encoding='utf8', errors='backslashreplace')
  procon_train = pd.read_csv(proconfile_train, sep=',', header=0)


  # Create a copy and backtranslate the tweets
  procon_train_copy_CasualFormal = procon_train.copy()
  procon_train_copy_FormalCasual = procon_train.copy()
  procon_train_copy_ActivePassive = procon_train.copy()
  procon_train_copy_PassiveActive = procon_train.copy()
  for i, row in tqdm(procon_train.iterrows(), total=procon_train.shape[0]):
    procon_train_copy_CasualFormal.at[i, "Query"] = transferStyle(row["Query"], sf_CasualFormal)
    procon_train_copy_FormalCasual.at[i, "Query"] = transferStyle(row["Query"], sf_FormalCasual)
    procon_train_copy_ActivePassive.at[i, "Query"] = transferStyle(row["Query"], sf_ActivePassive)
    procon_train_copy_PassiveActive.at[i, "Query"] = transferStyle(row["Query"], sf_PassiveActive)

  procon_style_transer = pd.concatenate([procon_train_copy_CasualFormal,procon_train_copy_FormalCasual,procon_train_copy_ActivePassive,procon_train_copy_PassiveActive], ignore_index=True)
# Save the Modified Dataset to Drive
#procon_train_copy_CasualFormal.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Procon20/modified/train_Casual_To_Formal.csv', index=False)
#procon_train_copy_FormalCasual.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Procon20/modified/train_Formal_To_Casual.csv', index=False)
#procon_train_copy_ActivePassive.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Procon20/modified/train_Active_To_Passive.csv', index=False)
#procon_train_copy_PassiveActive.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Procon20/modified/train_Passive_To_Active.csv', index=False)

# Open SemEval-2016
txtfile = open('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/SemEval-2016/data-all-annotations/trainingdata-all-annotations.txt', encoding='utf8', errors='backslashreplace')
semeval_train = pd.read_csv(txtfile, sep='\t')

# Create a copy and backtranslate the tweets
semeval_train_copy_CasualFormal = semeval_train.copy()
semeval_train_copy_FormalCasual = semeval_train.copy()
semeval_train_copy_ActivePassive = semeval_train.copy()
semeval_train_copy_PassiveActive = semeval_train.copy()
for i, row in tqdm(semeval_train.iterrows(), total=semeval_train.shape[0]):
  semeval_train_copy_CasualFormal.at[i, 'Tweet'] = transferStyle(row['Tweet'], sf_CasualFormal)
  semeval_train_copy_FormalCasual.at[i, 'Tweet'] = transferStyle(row['Tweet'], sf_FormalCasual)
  semeval_train_copy_ActivePassive.at[i, 'Tweet'] = transferStyle(row['Tweet'], sf_ActivePassive)
  semeval_train_copy_PassiveActive.at[i, 'Tweet'] = transferStyle(row['Tweet'], sf_PassiveActive)

semeval_style_transfer = pd.concatenate([semeval_train_copy_CasualFormal,semeval_train_copy_FormalCasual,semeval_train_copy_ActivePassive,semeval_train_copy_PassiveActive], ignore_index=True)
# Save the Modified Dataset to Drive
semeval_train_copy_CasualFormal.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/train_Casual_To_Formal.csv', index=False)
semeval_train_copy_FormalCasual.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/train_Formal_To_Casual.csv', index=False)
semeval_train_copy_ActivePassive.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/train_Active_To_Passive.csv', index=False)
semeval_train_copy_PassiveActive.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/train_Passive_To_Active.csv', index=False)

Mounted at /content/drive


0it [00:00, ?it/s]

  0%|          | 0/2814 [00:00<?, ?it/s]

[None]
['Lord thank you for your blessing forgive my sin give me strength and energy for this busy day ahead #blessed #hope #semST']
['ur blessings forgive my sins will be given by dear lord me strength and energy for this busy day ahead #blessed #hop']
['dear lord thank u for all of ur blessings forgive my sins lord give me strength and energy for this busy day ahead']
['Blessed are the peacemakers, they will be called children of God.', 'Matthew 5:9 #scripture #peace #semST.']
['peacemakers shall be called children of god.', 'Matthew 5:9 #scripture #peace #semST']
['Blessed are the peacemakers, for they shall be called children of God', 'for Matthew 5:9 #scripture #peace #SemST']
['Blessed are the peacemakers, for they shall be called children of God', 'Matthew 5:9 #scripture #peace #SemST']
["I'm not conformed to this world.", 'I am transformed by the renewing of my mind.', '#ISpeakLife #God 2014 #SemST.']
["I'm not conformed to this world.", 'by renewing my mind it changes me', 'is

In [ ]:
if usingProcon:
  final_styletransfer = pd.concat([procon_style_transer,semeval_style_transfer], ignore_index=True)
  final_styletransfer.to_csv(f'/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/modifiedCombined/train_styletransfer.csv')
else:
  semeval_style_transfer.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/train_styletransfer_final.csv', index=False)